In [3]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.0
    Uninstalling numpy-1.25.0:
      Successfully uninstalled numpy-1.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.11 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.6 requires jax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b709eb09a8886f3f0e3be1dfe64fed5bd8fe442a6cf3df9f5541d6fd2284eed4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [4]:
import os, glob
import logging

from pprint import pprint
from tqdm import tqdm
import json
# import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅한다
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# The percentage of the dataset you want to split as train and test
TEST_SIZE = 0.1

MAX_INPUT_LENGTH = 512  # Maximum length of the input to the Encoder
MIN_TARGET_LENGTH = 5  # Minimum length of the output by Decoder
MAX_TARGET_LENGTH = 128  # Maximum length of the output by Decoder
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# HF model hub에서 가져올 모델 이름
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

# 데이터 불러오기

In [6]:
from datasets import Dataset, load_dataset

In [7]:
ds_dir1 = './drive/MyDrive/Datasets/Texts/09.literature/20per/'
ds_dir2 = './drive/MyDrive/Datasets/Texts/09.literature/2~3sent/'
ds_path = glob.glob(ds_dir1 + '*.json') + glob.glob(ds_dir2 + '*.json')
len(ds_path)

9600

In [8]:
doc_id, document, summary = [], [], []

for path_ in tqdm(ds_path):
    # json 파일을 열어 필요한 정보 추출하기
    with open(path_, 'r') as f:
        # load와 loads를 잘 구별해서 사용하자
        doc = json.load(f)

        doc_id.append(doc["Meta(Acqusition)"]["doc_id"])
        document.append(doc['Meta(Refine)']['passage'])
        summary.append(doc['Annotation']['summary1'])
# Dataset.from_dict를 이용해 HF Dataset클래스로 만들기
raw_datasets = Dataset.from_dict({'id': doc_id,
                                  'document': document,
                                  'summary': summary})

100%|██████████| 9600/9600 [19:58<00:00,  8.01it/s]


In [9]:
# 확인 1
pprint(raw_datasets[1])

{'document': ' 그 세상에 가 있는 사람 외에는 거개 타락하여 일개의 시민이 되거나 그렇 지 않으면 표변하여 버렸거나 하였다. 그 '
             '중에서 양심을 버리지 않는 사람 이 어느 결엔지 바다를 건너 날쌔게 달아났다. 당시에는 갈 바를 몰라 마음 이 '
             '설레던 것도 때를 지남을 따라 초조의 속에서도 차차 마음이 가라 앉았 다. 반년 동안이나 우물쭈물 지나는 동안에 '
             '그는 알맞은 사람을 얻어 잡지 를 시작하게 되었다. 물론 그것이 마지막 목적은 아니었으나 그럭저럭 하는 동안에 '
             '마음의 안정도 얻고 한편으로 시세도 살피자는 뜻이었다. 그러나 일년도 지탱하지 못하고 잡지는 실패였다. 끌어댄 '
             '친구는 가엾게도 얼마 안되는 자본을 완전히 소탕하여 버렸다. 그마저 없어지니 건은 입에 풀칠할 도리조차 없어 가난과 '
             '불안의 구렁 속에서 헤매일 수밖에 없었다.\n'
             '카페의 여급으로 있는 보배를 알게 되고 가까워진 것은 이런 때였다. 건은 보배를 원하였고 보배는 건을 구하였다. '
             '반드시 연애가 아닌 것도 아니었으 나 보배가 건을 구한 것인 그 역 당시 마음의 가난과 불만이 있었기 때문이 었다. '
             '보배는 그때에 실연의 괴롬과 상처가 아직 온전히 사라지지 않은 중이었 다. 학교 시대의 스승이요, 학교를 나와서는 '
             '애인이라고 믿었던 사람이 사 랑의 유물까지 남긴 뒤 하필 사람이 없어 그의 동창의 동무를 이끌고 달아 난 것이었다. '
             '생각하여 보면 한 사람의 불량한 스승이 장기인 음악을 낚시 삼아 두 사람의 제자를 교묘하게 차례차례로 낚은 '
             '셈이었다.\n',
 'id': 'REPORT-literature-08626',
 'summary': '카페의 여급으로 있는 보배는 

In [10]:
# 확인 2
pprint(raw_datasets)

Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 9600
})


In [11]:
# 학습데이터와 테스트 데이터를 분리 Dataset obj > DatasetDict obj
raw_datasets = raw_datasets.train_test_split(test_size=TEST_SIZE)

In [12]:
pprint(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'summary'],
        num_rows: 8640
    })
    test: Dataset({
        features: ['id', 'document', 'summary'],
        num_rows: 960
    })
})


# 데이터 전처리

## 토크나이저 불러오기

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

## prefix 생성

In [14]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b", "psyche/KoT5-summarization"]:
    prefix = "요약: "
else:
    prefix = ""

In [15]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [16]:
# input_ids, attention_mask, labels를 추가한다.
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
pprint(tokenized_datasets)

Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8640
    })
    test: Dataset({
        features: ['id', 'document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 960
    })
})


In [22]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
# 요약 task는 seq2seq task로 분류된다

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, from_pt=True)
# 원본이 pytorch 이므로 from_pt=True

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')

In [19]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    drop_remainder=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    drop_remainder=True,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        drop_remainder=True,
        collate_fn=data_collator,
    )
)

In [93]:
type(generation_dataset.unbatch())

tensorflow.python.data.ops.unbatch_op._UnbatchDataset

In [71]:
first_batch = next(iter(train_dataset.batch(1)))
print(first_batch['labels'].shape)

(1, 8, 39)


In [72]:
first_batch

{'input_ids': <tf.Tensor: shape=(1, 8, 474), dtype=int64, numpy=
 array([[[16038, 26202,  2563, ...,     0,     0,     0],
         [16038, 26202,  7127, ...,     0,     0,     0],
         [16038, 26202,   194, ..., 25892, 25889,     1],
         ...,
         [16038, 26202, 25889, ...,     0,     0,     0],
         [16038, 26202, 10392, ...,     0,     0,     0],
         [16038, 26202,  2504, ...,     0,     0,     0]]])>,
 'attention_mask': <tf.Tensor: shape=(1, 8, 474), dtype=int64, numpy=
 array([[[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 1, 1, 1],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]])>,
 'labels': <tf.Tensor: shape=(1, 8, 39), dtype=int64, numpy=
 array([[[  174, 25269,  2765,  2043, 25894, 13108, 14284, 25936,  9364,
           2563, 21525,   809, 25890, 26514, 10308,    92,  1812,    36,
            441,  1039,    89, 25975, 15245,   809, 25895,  5331,    3

In [94]:
for idx, element in enumerate(generation_dataset.unbatch()):
    if idx == 1:
        print(element['input_ids'])
        print()
        print(element['input_ids'].numpy())
        print()
        print(tf.io.serialize_tensor(element['input_ids']))
        print()
        serialized_element = tf.io.serialize_tensor(element['input_ids']).numpy()
        print(serialized_element)
print('완료')

InternalError: ignored

# TFRecords 포맷으로 저장

In [20]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_features(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    serialized_nonscalar = tf.io.serialize_tensor(value)
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_nonscalar.numpy()]))

In [21]:
def make_example(input_ids, attention_mask, labels):
    # Create a dictionary with features that may be relevant.
    feature = {'input_ids':_int64_features(input_ids),
               'attention_mask':_int64_features(attention_mask),
               'labels':_int64_features(labels)}

    return tf.train.Example(features=tf.train.Features(feature=feature))

## 실험장

In [ ]:
nonscalar = [1, 2]

serialized_nonscalar = tf.io.serialize_tensor(nonscalar)
# tf.io.parse_tensor(serialized_nonscalar, out_type=tf.int32)

feature_of_bytes = tf.train.Feature(
  bytes_list=tf.train.BytesList(value=[serialized_nonscalar.numpy()]))

features_for_example = {
  'feature0': feature_of_bytes
}

example_proto = tf.train.Example(
  features=tf.train.Features(feature=features_for_example))

example_proto

features {
  feature {
    key: "feature0"
    value {
      bytes_list {
        value: "\010\003\022\004\022\002\010\002\"\010\001\000\000\000\002\000\000\000"
      }
    }
  }
}

In [ ]:
serialized_nonscalar

<tf.Tensor: shape=(), dtype=string, numpy=b'\x08\x03\x12\x04\x12\x02\x08\x02"\x08\x01\x00\x00\x00\x02\x00\x00\x00'>

In [ ]:
tf.io.parse_tensor(serialized_nonscalar, out_type=tf.int32)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [ ]:
feature_of_bytes

bytes_list {
  value: "\010\003\022\004\022\002\010\002\"\010\001\000\000\000\002\000\000\000"
}

In [ ]:
features_for_example

{'feature0': bytes_list {
   value: "\010\003\022\004\022\002\010\002\"\010\001\000\000\000\002\000\000\000"
 }}

In [ ]:
example = tf.io.parse_single_example(example_proto.SerializeToString(), {'feature0':tf.io.FixedLenFeature([], tf.string)})
example

{'feature0': <tf.Tensor: shape=(), dtype=string, numpy=b'\x08\x03\x12\x04\x12\x02\x08\x02"\x08\x01\x00\x00\x00\x02\x00\x00\x00'>}

In [108]:
t = example['feature0'].numpy()
# t = tf.cast(t, tf.string)
t.decode('utf-8')

'사과'

In [ ]:
tf.io.parse_tensor(t, out_type=tf.int32)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [ ]:
_int64_features(tokenized_datasets['train']['labels'][0])

bytes_list {
  value: "\010\003\022\004\022\002\010\031\"d\232\017\000\000\262\013\000\0003\035\000\000\337\014\000\000ue\000\000:e\000\000\357\037\000\0001e\000\000\252\000\000\000Be\000\000\261\025\000\000\304\003\000\000!g\000\000\033\000\000\000\301i\000\000\341e\000\000<e\000\000\004\013\000\000K\005\000\000<e\000\000\177\010\000\000\034f\000\000\177\005\000\000$e\000\000\001\000\000\000"
}

In [75]:
feature_of_bytes = _bytes_feature('사과'.encode('utf-8'))
feature_of_bytes

bytes_list {
  value: "\354\202\254\352\263\274"
}

In [77]:
features_for_example = {
  'feature0': feature_of_bytes
}
features_for_example

{'feature0': bytes_list {
   value: "\354\202\254\352\263\274"
 }}

In [80]:
example_proto = tf.train.Example(
  features=tf.train.Features(feature=features_for_example))

example_proto

features {
  feature {
    key: "feature0"
    value {
      bytes_list {
        value: "\354\202\254\352\263\274"
      }
    }
  }
}

In [81]:
example = tf.io.parse_single_example(example_proto.SerializeToString(), {'feature0':tf.io.FixedLenFeature([], tf.string)})
example

{'feature0': <tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x82\xac\xea\xb3\xbc'>}

In [85]:
t = example['feature0']
t = tf.cast(t, tf.string)
t

<tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x82\xac\xea\xb3\xbc'>

In [89]:
tensor1 = tf.strings.unicode_decode(t, 'utf-8')
tensor1

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([49324, 44284], dtype=int32)>

In [101]:
type(tensor1)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
for idx, x in enumerate(tokenized_datasets['test']):
    if idx == 2:
        break
    print(x['id'])

REPORT-literature-03709
REPORT-literature-01547


## 저장하기

In [22]:
os.makedirs('./tfrecord', exist_ok=True)

In [73]:
print("Writing tfrecord file...")
output_path = './tfrecord/KoT5_gen.tfrecord'
with tf.io.TFRecordWriter(output_path) as writer:
    data_iterator = iter(generation_dataset.unbatch())

    while True:
        try:
            data = next(data_iterator)
            tf_example = make_example(input_ids=data['input_ids'], attention_mask=data['attention_mask'], labels=data['labels'])
            writer.write(tf_example.SerializeToString())
        except StopIteration:
            break

Writing tfrecord file...


In [74]:
# list에 담긴 모든 정보들을 형 변환해서 tfrecords로 저장
print("Writing tfrecord file...")
output_path = './tfrecord/KoT5_train.tfrecord'
with tf.io.TFRecordWriter(output_path) as writer:
    data_iterator = iter(train_dataset.unbatch())

    while True:
        try:
            data = next(data_iterator)
            tf_example = make_example(input_ids=data['input_ids'], attention_mask=data['attention_mask'], labels=data['labels'])
            writer.write(tf_example.SerializeToString())
        except StopIteration:
            break


Writing tfrecord file...


In [75]:
print("Writing tfrecord file...")
output_path = './tfrecord/KoT5_test.tfrecord'
with tf.io.TFRecordWriter(output_path) as writer:
    data_iterator = iter(test_dataset.unbatch())

    while True:
        try:
            data = next(data_iterator)
            tf_example = make_example(input_ids=data['input_ids'], attention_mask=data['attention_mask'], labels=data['labels'])
            writer.write(tf_example.SerializeToString())
        except StopIteration:
            break

Writing tfrecord file...


# TFRecords 데이터 확인하기

In [1]:
from google.colab import auth
auth.authenticate_user()

In [23]:
def _parse_function(example_proto):

    name_to_features = {'input_ids': tf.io.FixedLenFeature([], tf.string),
                        'attention_mask':tf.io.FixedLenFeature([], tf.string),
                        'labels':tf.io.FixedLenFeature([], tf.string)}

    example = tf.io.parse_single_example(example_proto, name_to_features)

    for name in list(example.keys()):
        t = example[name]
        example[name] = tf.io.parse_tensor(t, out_type=tf.int64)

    return example

In [25]:
def load_tfrecord_dataset(tfrecord_name, batch_size, shuffle=True, buffer_size=10240):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()

    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)

    dataset = raw_dataset.map(
        _parse_function,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )


    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [26]:
tfr_path = 'gs://ohsori-tfrecord/tfrecord/KoT5_train.tfrecord'
train = load_tfrecord_dataset(tfr_path, batch_size=BATCH_SIZE, shuffle=False)
print(type(train))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [27]:
tfr_path = 'gs://ohsori-tfrecord/tfrecord/KoT5_test.tfrecord'
test = load_tfrecord_dataset(tfr_path, batch_size=BATCH_SIZE, shuffle=False)
print(type(test))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [28]:
tfr_path = 'gs://ohsori-tfrecord/tfrecord/KoT5_gen.tfrecord'
gen = load_tfrecord_dataset(tfr_path, batch_size=BATCH_SIZE, shuffle=False)
print(type(gen))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [29]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.TPUStrategy(tpu)

print(f"Available number of replicas: {strategy.num_replicas_in_sync}")

Available number of replicas: 8


In [30]:
train

<_PrefetchDataset element_spec={'attention_mask': TensorSpec(shape=<unknown>, dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=<unknown>, dtype=tf.int64, name=None), 'labels': TensorSpec(shape=<unknown>, dtype=tf.int64, name=None)}>

In [31]:
first_batch = next(iter(train.batch(1)))
print(first_batch['input_ids'].shape)

(1, 8, 512)


In [32]:
import tensorflow as tf

# Assuming you have a PrefetchDataset with unknown shape tensors


# Define the expected shapes of your tensors
input_ids_shape = tf.TensorShape([BATCH_SIZE, None])  # Shape [None] means the dimension can be of any size
attention_mask_shape = tf.TensorShape([BATCH_SIZE, None])
labels_shape = tf.TensorShape([BATCH_SIZE, None])

# Apply the shapes to the tensors in the dataset
new_train = train.map(lambda data: {
    'input_ids': tf.ensure_shape(data['input_ids'], input_ids_shape),
    'attention_mask': tf.ensure_shape(data['attention_mask'], attention_mask_shape),
    'labels': tf.ensure_shape(data['labels'], labels_shape),
})
new_test = test.map(lambda data: {
    'input_ids': tf.ensure_shape(data['input_ids'], input_ids_shape),
    'attention_mask': tf.ensure_shape(data['attention_mask'], attention_mask_shape),
    'labels': tf.ensure_shape(data['labels'], labels_shape),
})
new_gen = gen.map(lambda data: {
    'input_ids': tf.ensure_shape(data['input_ids'], input_ids_shape),
    'attention_mask': tf.ensure_shape(data['attention_mask'], attention_mask_shape),
    'labels': tf.ensure_shape(data['labels'], labels_shape),
})

In [33]:
new_train

<_MapDataset element_spec={'input_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(8, None), dtype=tf.int64, name=None)}>

In [34]:
new_train = new_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
new_test = new_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
new_gen = new_gen.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [95]:
first_batch = next(iter(new_train.batch(1)))

In [96]:
print(first_batch['input_ids'].shape)

(1, 8, 512)


In [97]:
first_batch

{'input_ids': <tf.Tensor: shape=(1, 8, 512), dtype=int64, numpy=
 array([[[16038, 26202, 18596, ...,     0,     0,     0],
         [16038, 26202,  8874, ...,     0,     0,     0],
         [16038, 26202,   438, ...,     0,     0,     0],
         ...,
         [16038, 26202, 25889, ...,     0,     0,     0],
         [16038, 26202,   601, ...,     0,     0,     0],
         [16038, 26202, 25889, ...,     0,     0,     0]]])>,
 'attention_mask': <tf.Tensor: shape=(1, 8, 512), dtype=int64, numpy=
 array([[[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]])>,
 'labels': <tf.Tensor: shape=(1, 8, 39), dtype=int64, numpy=
 array([[[  504, 25938,    68,  3363,    61,  5457,   851,  5432,   172,
          14699,  6705,  6197,   100, 25965, 26074, 25905,   504, 26056,
          26402, 25894,  3975,   504, 25895, 16915,  5407, 25892,     

## 모델 학습 시켜보기

In [35]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [36]:
from tensorflow.keras.backend import clear_session
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
# 요약 task는 seq2seq task로 분류된다

clear_session()

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [37]:
optimizer = keras.optimizers.AdamW(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer) # loss 값은 내부적으로 처리하도록 설정

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [38]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  84954240  
                                                                 
 decoder (TFT5MainLayer)     multiple                  113275008 
                                                                 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.backend import clear_session

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=new_gen, predict_with_generate=True
)

callbacks = [metric_callback]


# For now we will use our test set as our validation_data
model.fit(
    new_train, validation_data=new_test,
    epochs=MAX_EPOCHS,
    callbacks=callbacks,
    steps_per_epoch=8640//BATCH_SIZE,
    validation_steps=960//BATCH_SIZE,
)

1022/1080 [===========================>..] - ETA: 6:16 - loss: 1.8817